In [1]:
import findspark
findspark.add_jars('/app/postgresql-42.1.4.jar')
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("argentinaDatosAdd")
    .config("spark.driver.memory", "512m")
    .config("spark.driver.cores", "1")
    .config("spark.executor.memory", "512m")
    .config("spark.executor.cores", "1")
    .config("spark.sql.shuffle.partitions", "2")
    .getOrCreate()
)

In [3]:
from pyspark.sql.functions import *

In [4]:
pas = 'p4ssW0rdP4r4Agr1cultur4'

In [5]:
df = spark.read.format("jdbc") \
.option("url", "jdbc:postgresql://postgres/agricultura") \
.option("driver", "org.postgresql.Driver") \
.option("dbtable", "agricultura.argentina_datos") \
.option("user", "agricultura") \
.option("password", pas) \
.load()

In [6]:
provRindeCultivos = (
df.groupBy('provincia', 'cultivo', 'campania', 'fecha')
.agg(avg('rendimiento_kgha').alias('rinde_kxha'))
.orderBy(desc('rinde_kxha'))
)

In [7]:
provRindeCultivos \
    .write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/agricultura") \
    .option("dbtable", "agricultura.prov_rinde") \
    .option("user", "agricultura") \
    .option("password", pas) \
    .option("driver", "org.postgresql.Driver") \
    .mode('overwrite') \
    .save()

In [8]:
sojaBArinde = (
df.filter(df['provincia'] == "BUENOS AIRES")
.filter(df['cultivo'] == 'Soja total')
.select('provincia','rendimiento_kgha', 'campania', 'fecha')
)

In [9]:
sojaBArinde \
    .write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/agricultura") \
    .option("dbtable", "agricultura.soja_ba_rinde") \
    .option("user", "agricultura") \
    .option("password", pas) \
    .option("driver", "org.postgresql.Driver") \
    .mode('overwrite') \
    .save()

In [10]:
cultivosProvincias = (
df.withColumn("tn_cosechadas",(col("ha_cosechada") * col('rendimiento_kgha'))/1000)
.groupBy('cultivo', 'provincia', 'fecha')
.agg(sum('tn_cosechadas').alias('tn_total'))
.orderBy(desc('tn_total'))
)

In [11]:
cultivosProvincias \
.write \
.format('jdbc') \
.option('url', 'jdbc:postgresql://postgres/agricultura') \
.option('dbtable', 'agricultura.cultivos_provincias') \
.option('user', 'agricultura') \
.option('password', pas) \
.option('driver', 'org.postgresql.Driver') \
.mode('overwrite') \
.save()

In [12]:
spark.stop()